In [2]:
%%bash

# mkdir ~/.qlib
# mkdir ~/.qlib/qlib_data
# mkdir ~/.qlib/qlib_data/cn_data
# mkdir ~/.qlib/qlib_data/cn_data/calendars
# mkdir ~/.qlib/qlib_data/cn_data/instruments
# mkdir ~/.qlib/qlib_data/cn_data/cn_1d_norm

In [3]:
import pandas as pd

In [4]:
import wk_db

wk_db.read_sql('select count(*) from a_share_market', db_loc='low_freq_db')

,count(*)
0,11675916


In [5]:
dt = wk_db.read_sql('SELECT DISTINCT trade_dt FROM a_share_market ORDER BY trade_dt;', db_loc='low_freq_db')

In [6]:
dt.to_csv('~/.qlib/qlib_data/cn_data/calendars/day.txt', index=False, header=False)

In [7]:
symbol_wk = wk_db.read_sql('SELECT DISTINCT windcode FROM a_share_market ORDER BY windcode;', db_loc='low_freq_db')

In [8]:
symbol_wk.head()

,windcode
0,000001.SZ
1,000002.SZ
2,000004.SZ
3,000005.SZ
4,000006.SZ


In [15]:
def symbol_wk2qlib(s, lower=True):
    code, mkt = s.split('.')
    if lower:
        res = mkt.lower() + code
    else:
        res = mkt.upper() + code
    return res

For WK: norm_price = raw_price * adj_factor / (ori_raw_price * ori_adj_factor)

In QLIB: norm_price = raw_price * factor

In [10]:
# for _, row in symbol_wk.iterrows():
#     s = row['windcode']
#     print(s)
#     # df = wk_db.read_sql("SELECT trade_dt, windcode, open_price, high_price, low_price, close_price, volume, amount, adj_factor FROM a_share_market WHERE windcode='{s}';".format(s=s), db_loc='low_freq_db')
#     df = wk_db.read_sql("SELECT * FROM a_share_market WHERE windcode='{s}' ORDER BY trade_dt;".format(s=s), db_loc='low_freq_db')
#     ori_raw_price = df.iloc[0]['close_price']
#     ori_adj_factor = df.iloc[0]['adj_factor']
#     _df = pd.DataFrame()
#     _df['date'] = df['trade_dt']
#     _df['symbol'] = df['windcode'].apply(symbol_wk2qlib)
#     _df['factor'] = df['adj_factor'] / (ori_raw_price*ori_adj_factor)
#     _df['open'] = df['open_price'] * _df['factor']
#     _df['open'] = df['open_price'] *  _df['factor']
#     _df['high'] = df['high_price'] * _df['factor']   
#     _df['low'] = df['low_price'] * _df['factor']
#     _df['close'] = df['close_price'] *  _df['factor']
#     _df['volume'] = df['volume']
#     _df['amount'] = df['amount'] *  _df['factor']
#     # print(df)
#     # print(_df)
#     wk_symbol = symbol_wk2qlib(s)
#     _df.to_csv(f'~/.qlib/qlib_data/cn_data/cn_1d_norm/{wk_symbol}.csv', index=False)
#     # break

In [11]:
all_code = wk_db.read_sql('SELECT DISTINCT windcode FROM a_share_market;', db_loc='low_freq_db')
all_code

,windcode
0,002064.SZ
1,002150.SZ
2,002225.SZ
3,600397.SH
4,601231.SH
...,...
5583,301517.SZ
5584,688653.SH
5585,301139.SZ
5586,301606.SZ


In [23]:
all_code = wk_db.read_sql('SELECT windcode, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM a_share_market GROUP BY windcode ORDER BY windcode;', db_loc='low_freq_db')
all_code['windcode'] = all_code['windcode'].apply(lambda s: symbol_wk2qlib(s, False))
all_code

,windcode,st,ed
0,SZ000001,2010-01-04,2024-10-31
1,SZ000002,2010-01-04,2024-10-31
2,SZ000004,2010-01-04,2024-10-31
3,SZ000005,2010-01-04,2024-04-26
4,SZ000006,2010-01-04,2024-10-31
...,...,...,...
5583,BJ920019,2024-10-11,2024-10-31
5584,BJ920066,2024-10-31,2024-10-31
5585,BJ920088,2024-10-29,2024-10-31
5586,BJ920099,2024-09-25,2024-10-31


In [24]:
all_code.to_csv(f'~/.qlib/qlib_data/cn_data/instruments/all.txt', index=False, header=False, sep='\t')

In [25]:
idx_map = {
    'csi300': "000300.SH"
}

In [26]:
idx_name = 'csi300'
idx_code = idx_map[idx_name]

idx_df = wk_db.read_sql(f"SELECT con_windcode, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM index_components_weight WHERE index_windcode='{idx_code}' GROUP BY con_windcode ORDER BY con_windcode;", db_loc='low_freq_db')
idx_df['con_windcode'] = idx_df['con_windcode'].apply(lambda s: symbol_wk2qlib(s, False))
idx_df

,con_windcode,st,ed
0,SZ000001,2010-01-04,2024-10-31
1,SZ000002,2010-01-04,2024-10-31
2,SZ000008,2016-12-30,2018-06-01
3,SZ000009,2010-01-04,2017-12-01
4,SZ000012,2010-01-04,2014-12-01
...,...,...,...
768,SH688363,2021-06-30,2024-10-31
769,SH688396,2021-06-30,2024-10-31
770,SH688561,2021-12-31,2024-06-03
771,SH688599,2021-12-31,2024-10-31


In [27]:
idx_df.to_csv(f'~/.qlib/qlib_data/cn_data/instruments/{idx_name}.txt', index=False, header=False, sep='\t')